In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("../data/raw/train_dataset.csv")
df = data.copy(deep = True)
df

In [ ]:
df.loc[df['eyesight(left)'] == 9.9, 'eyesight(left)'] = 0
df.loc[df['eyesight(right)'] == 9.9, 'eyesight(right)'] = 0

df

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
df.info()
df.describe()

In [ ]:
import smoker_stat_pred.plots as custom_plots
fig = custom_plots.plot_histogram(df, 'eyesight(left)')
fig.show()

In [ ]:
fig = custom_plots.plot_histogram(df, 'eyesight(right)')
fig.show()

In [ ]:
# hearing
def merge_hearing(row):
    if row['hearing(left)'] == 1 and row['hearing(right)'] == 1:
        return 0   # normal in both ears
    elif row['hearing(left)'] == 2 and row['hearing(right)'] == 2:
        return 2   # abnormal in both ears
    else:
        return 1   # abnormal in one ear only

df['hearing'] = df.apply(merge_hearing, axis=1)
df.drop(columns = ['hearing(left)', 'hearing(right)'], inplace = True)
df

In [ ]:
fig = custom_plots.plot_histogram(df, 'hearing')
fig.show()

In [ ]:
df.drop(columns=["hearing"], inplace=True)

In [ ]:
fig = custom_plots.plot_correlation_matrix(df)
fig.show()

In [ ]:
df

In [ ]:
df["BMI"] = df["weight(kg)"] / ((df["height(cm)"] / 100) ** 2)

In [ ]:
df.drop(columns=['height(cm)', 'weight(kg)'], inplace=True)
df

In [ ]:
fig = custom_plots.plot_histogram(df, 'Urine protein')
fig.show()

In [ ]:
fig = custom_plots.plot_histogram(df, 'hemoglobin')
fig.show()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
fig = custom_plots.plot_correlation_matrix(df)
fig.show()

In [ ]:
df.drop(columns=['waist(cm)'], inplace=True)
df.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (20,10))
sns.heatmap(df.corr(),annot=True , cmap ='YlOrRd' )
plt.title("Correlation of Features")

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = df.columns

vif_data["VIF"] = [variance_inflation_factor(df.values, i)
                          for i in range(len(df.columns))]

vif_data

In [ ]:
# normalizing
cols_to_norm = df.columns
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df

In [ ]:
d = pd.DataFrame(df, columns=df.columns)
d.to_csv("../data/processed/normalized_df.csv", index=False)